In [ ]:
import os
from io import StringIO

import sys

current_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'PVT-thermal-performance-model')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'PVT-thermal-performance-model', 'CFD')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'RD-systems-and-test-benches')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'parallel-flow-distribution-pressure-loss')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'parallel-flow-distribution-pressure-loss', 'ansys')))

import numpy as np
import pandas as pd
import openpyxl

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import jou_gen as jg
import bridge_module as bm

import ansys.fluent.core as pyfluent

from CoolProp.CoolProp import PropsSI

import hx_hydraulic as hxhy

In [2]:
computer = 'lmps_cds'
jg.set_computer(computer)
server_number = 1104

category_folder_name = 'Hydraulics'
project_folder_name = 'NUMAIRO_motif'

caoMeshFolder = 'V4.6_1MPE_M1'
title = 'abaque0_laminar_try0'

nb_it = 500

report_names = ['report_distrib_inlet_sp',
                'report_distrib_outlet_sp',
                'report_coll_inlet_sp',
                'report_coll_outlet_sp',
                'report_distrib_inlet_mdot',
                'report_distrib_outlet_mdot',
                'report_coll_inlet_mdot',
                'report_coll_outlet_mdot'
                ]

In [ ]:
if computer == 'seagull':
    root_fp = r'D:\ANSYS Fluent Projects'
else:
    root_fp = "/usrtmp/delachaux/ANSYS Fluent Projects"
    jg.computer = computer

main_fp = os.path.join(root_fp, category_folder_name, project_folder_name)
Mesh_fp = os.path.join(main_fp, 'mesh')
SR_fp = bm.check_folder(os.path.join(main_fp, 'SimulationResults'))
caoMesh_fp = bm.check_folder(os.path.join(SR_fp, caoMeshFolder))
param_fp = bm.check_folder(os.path.join(SR_fp, caoMeshFolder, title))

tui, solver = bm.init_solver(main_fp, str(server_number), subfolder='')
print('main_fp :',main_fp)

In [4]:
water_dict = solver.setup.materials.fluid["water-liquid"].get_state()

T_op = 23 + 273.15
p_op = 101325

if computer == 'lmps_cds':
    constant_key = 'value'
    water_dict['chemical_formula'] = ""
else:
    constant_key = 'constant'

water_dict['density']['option'] = 'constant'
water_dict['density'][constant_key] = PropsSI('D', 'T', T_op, 'P', p_op, 'INCOMP::MPG[0.4]')
water_dict['viscosity']['option'] = 'constant'
water_dict['viscosity'][constant_key] = PropsSI('viscosity', 'T', T_op, 'P', p_op, 'INCOMP::MPG[0.4]')

solver.setup.materials.fluid["water-glycol"].set_state(water_dict)

In [ ]:
parametric_df = pd.DataFrame()
parametric_df['alpha'] = np.array(5*[0.05, 0.25, 0.5, 0.75, 0.95])
parametric_df['beta'] = np.array([5*[x] for x in np.linspace(0.1, 0.9, 5)]).flatten()
parametric_df['case'] = 'case' + parametric_df.index.astype(str)
parametric_df['Vdot_max'] = 100
parametric_df['mdot_max'] = (parametric_df['Vdot_max']/(3.6*1E6))*water_dict['density'][constant_key]
parametric_df['mdot_distrib_inlet'] = parametric_df['mdot_max']*parametric_df['beta']
parametric_df['mdot_coll_inlet'] = parametric_df['mdot_max'] - parametric_df['mdot_distrib_inlet']
parametric_df['mdot_distrib_outlet'] = parametric_df['mdot_distrib_inlet']*(1-parametric_df['alpha'])
parametric_df

In [ ]:
for index, row in parametric_df.iterrows():

    case_name = str(row['case'])
    folder_path_case = bm.check_folder(os.path.join(SR_fp, title, case_name))

    for report_name in report_names:
        jg.change_report_file_path(tui, report_name, os.path.join(folder_path_case, f"{report_name}.out"))

    jg.change_named_expression(tui, 'mdot_distrib_inlet', row['mdot_distrib_inlet'], 'kg/s')
    jg.change_named_expression(tui, 'mdot_coll_inlet', row['mdot_coll_inlet'], 'kg/s')
    jg.change_named_expression(tui, 'mdot_distrib_outlet', row['mdot_distrib_outlet'], 'kg/s')

    solver.solution.initialization.hybrid_initialize()

    if computer == 'seagull':
        solver.solution.run_calculation.iterate(number_of_iterations = nb_it)
    elif computer == 'lmps_cds':
        solver.solution.run_calculation.iterate(iter_count = nb_it)

    jg.write_report(tui, 'mdot', folder_path_case, 'mdot_report')
    jg.write_report(tui, 'sp', folder_path_case, 'sp_report')
    jg.write_residuals_file(tui, folder_path_case, 'residuals')
    jg.write_data(tui, folder_path_case, case_name)